___
# Ciência dos Dados - PROJETO 3 - INSPER

___
## Leonardo Bhering 

## Gustavo Molina

___

## INTRODUÇÃO

Este projeto tem como objetivo prever qual o valor de mercado de um jogador de futebol com base em algumas variaveis, como sua idade e seus números da temporada.

___
## MINERANDO DADOS e CARACTERÍSTICAS DO DATASET

[Detalhe aqui as características da base de dados além da análise descritiva.]

In [81]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
from numpy import arange
from sklearn.model_selection import train_test_split
pd.options.mode.chained_assignment = None

In [82]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\Gustavo\Downloads\cddp3-master


### BASE DE DADOS

In [83]:
dados = pd.read_excel('jogadores.xlsx')

In [84]:
dados.dtypes

Rank                                       int64
Jogador                                   object
Posição                                   object
Idade                                      int64
Valor de Mercado (em milhões de euros)     int64
Jogos                                      int64
Gols                                       int64
Assistencias                               int64
dtype: object

In [85]:
print('Base de dados com a qual trabalharemos:')
dados.head()

Base de dados com a qual trabalharemos:


,Rank,Jogador,Posição,Idade,Valor de Mercado (em milhões de euros),Jogos,Gols,Assistencias
0,1,Neymar,Extremo Esquerdo,26,180,32,29,19
1,2,Lionel Messi,Extremo Direito,30,180,55,45,18
2,3,Kylian Mbappé,Extremo Direito,19,120,50,23,19
3,4,Harry Kane,Ponta de Lança,24,120,48,41,5
4,5,Cristiano Ronaldo,Ponta de Lança,33,120,45,46,8


In [86]:
dados.columns = ['Rank','Jogador','posição','idade','valor','jogos','gols','assistencias']

In [87]:
dados_filtro = dados.iloc[0:,[2,3,4,5,6,7]]
dados_filtro.head()

,posição,idade,valor,jogos,gols,assistencias
0,Extremo Esquerdo,26,180,32,29,19
1,Extremo Direito,30,180,55,45,18
2,Extremo Direito,19,120,50,23,19
3,Ponta de Lança,24,120,48,41,5
4,Ponta de Lança,33,120,45,46,8


### ANÁLISE DESCRITIVA

In [88]:
#Dividindo a base de dados em 90% para treinamento e 10% para teste
treinamento, teste = train_test_split(dados_filtro, test_size=0.1)

In [89]:
teste.head()

,posição,idade,valor,jogos,gols,assistencias
50,Lateral Direito,26,60,48,1,8
56,Ponta de Lança,29,60,25,8,6
142,Médio Central,31,35,49,3,9
38,Ponta de Lança,25,65,49,16,6
101,Ponta de Lança,24,40,41,22,3


In [90]:
treinamento.head()

,posição,idade,valor,jogos,gols,assistencias
67,Defesa Central,23,50,33,3,0
199,Médio Central,24,25,51,8,3
13,Ponta de Lança,31,85,53,32,18
11,Ponta de Lança,29,90,50,42,5
4,Ponta de Lança,33,120,45,46,8


___
## MODELOS DE PREDIÇÃO

[O tópico C.PROCESSO E ESTATÍSTICAS DE VALIDAÇÃO estará dentro deste tópico. Organize-o aqui de forma clara!]

[ORGANIZE AQUI TODOS OS MODELOS DE PREDIÇÃO NECESSÁRIOS NESTE PROJETO!]

o MODELO DE PREDIÇÃO PELA MÉDIA

o MODELO DOS K VIZINHOS MAIS PRÓXIMOS (K-Nearest Neighbors Regression)

o MODELO DE REGRESSÃO LINEAR (Multiple Linear Regression)

o MODELO DE ÁRVORES DE REGRESSÃO (Decision Tree Regression)

Modelo de Predição pela Média:

In [91]:
#Modelo de Predição pela média
#Achando a média dos valores de mercado de cada jogador
media=treinamento['valor'].mean()
print(media)

49.266666666666666


In [92]:
#Aplicando o R^2
valor = (teste['valor'] - media)**2
r2 = 1 - (valor.sum()/valor.sum())
r2

0.0

In [93]:
#Aplicando o RMS
rms = (valor.sum()/20)**0.5
rms

15.57565764618339

Modelo de Regressão Linear:

In [94]:
#Separar em variáveis independentes e a variável dependente
X = dados_filtro.iloc[0:,[0,1,3,4,5]].values #Variáveis independentes
y = dados_filtro.iloc[0:,2].values #Variável dependente (Valor do jogador)

In [95]:
#Codificação da variável posição (de string para float)
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X = LabelEncoder()
X[:,0] = labelencoder_X.fit_transform(X[:,0])
onehotencoder = OneHotEncoder(categorical_features = [0])
X = onehotencoder.fit_transform(X).toarray()

In [96]:
X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.1, random_state = 0)

In [97]:
#Aplicando a Regressão Linear Multipla noa base trenamento
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(X_treino, y_treino)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [98]:
#Prevendo os resultados dos dados teste
y_pred = regressor.predict(X_teste)
y_pred

array([ 35.39064121,  35.4795506 ,  41.19645369,  44.30060816,
        42.75525974,  57.79411329,  63.17478808,  50.74069459,
        75.03398028,  52.3280977 ,  64.38758883,  39.91506359,
        49.12267669,  37.16136378,  44.01769797,  78.48187553,
        63.56390178,  43.7460045 ,  52.80041937,  45.06860621])

In [168]:
#Transformado o array(predição dos valores) e a coluna(teste['valor']) em lista
lista_y = y_pred.tolist()
lista_v = teste['valor'].tolist()

#For para calcular o valor da media dos valores de predição
media2 = 0
for i in range(len(lista_y)):
    media2 = media2 +lista_y[i]
media_f = media2/20
#calculando os valores da lista teste['valor'] - os valores da predição e elevando ao quadrado
lista_R = []
for j in range(len(lista_y)):
    R12 = (lista_v[j] - lista_y[j])**2
    lista_R.append(R12)
#Criando uma coluna no dataframe teste que contem os valores calculados anteriormente
teste['valor_estipulado'] = lista_R
#Somando todos os valores da coluna do dataframe adicionados anteriormente
div_f0 = teste['valor_estipulado'].sum()

#calculando os valores da lista teste['valor'] - os valores de media e elevando ao quadrado
lista_F = []
for l in range(len(lista_v)):
    div1 = (lista_v[l] - media_f)**2
    lista_F.append(div1)
#Transformando a lista_F em uma coluna do dataframe teste
teste['valor_estipulado_media'] = lista_F
#Somando todos os valores da coluna do dataframe adicionados anteriormente
div_f1 = teste['valor_estipulado_media'].sum()
#Fazendo a divisão para calcular o R2
R2f = 1 - (div_f0/div_f1)
print('O valor de R2 na regressão linear é {0}'.format(R2f))

O valor de R2 na regressão linear é -0.8594737499645146


In [164]:
#Calculando o RMS
RMS2 = (div_f0/20)**0.5
print('O valor de RMS na regressão linear é {0}'.format(RMS2))

O valor de RMS na regressão linear é 22.001611513713968


___
## CONCLUSÃO

___
## REFERÊNCIAS BIBLIOGRÁFICAS